Here i try using dask instead of pandas for a kaggle competition i entered.  There are a few different things.  First of all you can't use iloc to split a dataframe by index so it makes it very hard to use stratified split.  
But runtimes are much much quicker, so i suppose once you get used to it would be far superior for large datasets

It was quite difficult ot get kaggle into an amazon notebook, and there is not a lot of support for it, could be worth a blog post.

Also it was beyond me how to push this directly from aws.  

In [ ]:
!pip install tqdm

In [ ]:
!pip install kaggle

In [ ]:
!kaggle competitions download -c instant-gratification

In [10]:
import os

In [16]:
os.environ['KAGGLE_USERNAME'] = "****"

In [18]:
os.environ['KAGGLE_KEY'] = "****"

In [29]:
import pandas as pd
import numpy as np

In [33]:
test = pd.read_csv('test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

In [31]:
train = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

In [59]:
train=train.set_index('id',drop=True)
test=test.set_index('id',drop=True)

In [60]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
cols=[x for x in train.columns if x not in ['target','wheezy-copper-turtle-magic']]
scaler=StandardScaler()

In [61]:
magic_key=train.columns[146]
train_f=[train[train[magic_key]==i] for i in range(512)]
test_f=[test[test[magic_key]==i] for i in range(512)]
combo=[train_f[i][cols].append(test_f[i][cols]).copy() for i in range(512)]
N=11
qda=QuadraticDiscriminantAnalysis(0.5)
skf = StratifiedKFold(n_splits=N,random_state=75)
vt = VarianceThreshold(threshold=1.5)

In [65]:
#first of all we will correct the 'flipped' labels from the training data
%time
skfprobas={}
#for k in tqdm_notebook(range(2)):
for k in tqdm_notebook(range(512)):
    vt.fit(combo[k][cols])

    for train_index, test_index in skf.split(train_f[k][cols],train_f[k][['target']]):
        X_pl=vt.transform(train_f[k][cols].iloc[train_index])
        X_skf_test=vt.transform(train_f[k][cols].iloc[test_index])
        
        y_pl=train_f[k][['target']].iloc[train_index]
        
        qda.fit(X_pl,y_pl.values.ravel())
        y_preds_skf=qda.predict_proba(X_skf_test)
        temp_skf=train_f[k][['target']].iloc[test_index].copy()
        temp_skf['preds']=[x[1] for x in y_preds_skf]
        for kk in temp_skf.index:
            skfprobas[kk]=temp_skf['preds'][kk]

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 15.3 µs


In [66]:
%%time
import dask.dataframe as dd
from dask.distributed import Client
train_dask = dd.from_pandas(train, npartitions=3)
test_dask = dd.from_pandas(test, npartitions=3)
train_f_dask=[train_dask[train_dask[magic_key]==i] for i in range(512)]
test_f_dask=[test_dask[test_dask[magic_key]==i] for i in range(512)]
combo_dask=[train_f_dask[i][cols].append(test_f_dask[i][cols]).copy() for i in range(512)]
N=11
qda_dask=QuadraticDiscriminantAnalysis(0.5)
skf_dask = StratifiedKFold(n_splits=N,random_state=75)
vt_dask = VarianceThreshold(threshold=1.5)

CPU times: user 2min 27s, sys: 1.58 s, total: 2min 29s
Wall time: 2min 28s


In [78]:
import warnings
warnings.simplefilter('ignore')

In [82]:
%time
skfprobas_dask=[]
#for k in tqdm_notebook(range(2)):
for k in tqdm_notebook(range(512)):
    vt_dask.fit(combo_dask[k])
    X=vt_dask.transform(train_f_dask[k][cols])
    X_test=vt_dask.transform(test_f_dask[k][cols])
    y=train_f_dask[k][['target']]
    qda_dask.fit(X,y.values)
    preds = qda_dask.predict_proba(X_test)
    preds = [x[1] for x in preds]
    my_preds=test_f[k][test_f[k].columns[0]].copy()
    my_preds['preds']=preds
    skfprobas_dask.append(my_preds['preds'])

CPU times: user 6 µs, sys: 6 µs, total: 12 µs
Wall time: 24.6 µs


In [84]:
skfprobas_dask[0][:10]

[4.0907943231579687e-07,
 0.9999999312168185,
 4.4161858409879836e-24,
 0.9871257722919382,
 0.9999997492206025,
 0.9990180988196776,
 2.4144309667826135e-10,
 0.9999999999999374,
 0.999999999999305,
 0.022775279505056128]

In [85]:
1+1

2